In [ ]:
# This part you need to run once per project, which I have already done, so it is commented out. 
# It imports necessary databases (possibly a lengthy process), and backs up the project directory 
# so that it is saved in the home directory. The backed up project can be used by other project 
# collaborators without the need to import any databases again. The next section starts with restoring 
# the saved project from directory.

# import bw2data as bd
# import bw2calc as bc
# import bw2io as bi
# import numpy as np

# ei_path = "/Users/akim/Documents/LCA_files/ecoinvent_38_cutoff/datasets"
# ei_name = "ecoinvent 3.8 cutoff"

# bd.projects.set_current("Building paper")

In [ ]:
# # Import biosphere
# bi.bw2setup()

In [ ]:
# # Import ecoinvent
# if ei_name in bd.databases:
#     print("database already exists")
# else:
#     ei = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
#     ei.apply_strategies()
#     assert ei.all_linked
#     ei.write_database()

In [ ]:
# bi.backup_project_directory("Building paper")

In [1]:
import os
os.chdir("/home/shinder/0_work/1_Models/2_building/2a_integrations/model_material/lca")

import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np

# Only need to run this once! Otherwise you might work on this project, eg import `building lci` database, 
# but then overwrite it if you restore it from the backup
# fp_building_bw_project = "brightway2-project-Building paper-backup.19-September-2022-03-55PM.tar.gz"
# bi.restore_project_directory(fp_building_bw_project)

In [2]:
bd.projects.set_current("Building paper")
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	building lci
	ecoinvent 3.8 cutoff

In [139]:
# Import foreground database from excel

# delete database 
# -- doesn't hurt to do it every time you run the code, because it is very fast, but your database is then always updated
# del bd.databases["building lci"]  

# if "building lci" not in bd.databases:
fp_building_lci_excel = "building_lci_1_scenario_insulation_test.xlsx"
bld = bi.ExcelImporter(fp_building_lci_excel)
bld.apply_strategies()
bld.match_database("ecoinvent 3.8 cutoff", fields=("name", "location"))
bld.match_database()

if bld.all_linked:
    bld.write_database()
else:
    bld.statistics()

Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 15.36 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Applying strategy: link_iterable_by_fields
12 datasets
93 exchanges
0 unlinked exchanges
  


0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/26/2022 11:46:17
  Finished: 10/26/2022 11:46:17
  Total time elapsed: 00:00:00
  CPU %: 118.50
  Memory %: 0.19
Created database: building lci


In [123]:
list (bld.unlinked)

[]

In [140]:
# LCIA
bld = bd.Database("building lci")

n =12 #number of buildings
for x in range(1,n+1):
    building_name = str("building ")+str(x)
    demand_act = [act for act in bld if building_name in act['name']]
#     assert len(demand_act) == 1
    demand_act = demand_act[0]

    method = ('IPCC 2013', 'climate change', 'GWP 100a')

    lca = bc.LCA({demand_act: 1}, method)
    lca.lci()
    lca.lcia()
    print(building_name, lca.score)

building 1 5.225618930242147
building 2 7.996399843761052
building 3 3.0359596980818453
building 4 3.4709363406241196
building 5 2.2843791606999426
building 6 8.468499050672598
building 7 3.8210960315919875
building 8 3.5781753497420783
building 9 2.736259696839067
building 10 3.6410514076694094
building 11 4.1716641341421505
building 12 5.189325019046301


In [134]:
x = "stone wool"
ei =    bd.Database("ecoinvent 3.8 cutoff")
demand_act = [act for act in ei if x in act['name']]
print([act['type'] for act in ei if x in act['name']])
demand_act

['process', 'process', 'process', 'process', 'process', 'process', 'process', 'process', 'process']


['stone wool production' (kilogram, CH, None),
 'market for stone wool, packed' (kilogram, GLO, None),
 'market for stone wool factory' (unit, GLO, None),
 'stone wool factory construction' (unit, CH, None),
 'stone wool factory construction' (unit, RoW, None),
 'stone wool production, packed' (kilogram, RoW, None),
 'market for stone wool' (kilogram, GLO, None),
 'stone wool production, packed' (kilogram, CH, None),
 'stone wool production' (kilogram, RoW, None)]